# Flask web framework

Flask is a Python micro web framework created in 2010 (used by Pinterest and LinkedIn).  
The main difference with bigger web frameworks like Django is that its core is very minimalist : it does not come pre-packaged with a lot of functionalities, it does not require any other library, and allows granularity by importing only extensions for the features we need.

Flask has a lot of extensions offering various features such as validation, upload handling, authentication, database support...

### Installation

- Download the flask framework : `pip install flask`
- Ensure it is correctly installed : `flask --version`

### Hello World

To run a Flask webapp, we just need to create an instance of `Flask` and run it.  
To add routes to the webapp, we use the `app.route()` decorator.

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_handler():
    return 'Hello world'

if __name__ == '__main__':
    app.run(debug=True)

From a terminal, run it with `python app.py`, it should start the web server on port 5000 : http://127.0.0.1:5000/  
The `debug` mode allows to reload automatically the web server on code change.

### Path parameters

We can include som path parameters in the path of the decorator.  
The parameters name and type must be specified.  
The path parameters becomes available as function parameters in the handler function.

In [ ]:
@app.route('/users/<string:user_name>/posts/<int:post_id>')
def post_handler(user_name, post_id):
    return f'Fetching post ID {post_id} for user {user_name}'

### Restrict methods

We can restrict which HTTP methods are allowed for a given route.  
This is used a lot to allow GET and POST requests on a same route (GET to list the resources and POST to create one from a form for example).

In [ ]:
from flask import requests

@app.route('/users', methods=['GET', 'POST'])
def get_users():
    return 'Getting users with method %s' % request.method

The HTTP method of the request can be accessed via the `request` variable provided in the `flask` module.

### Templates

Instead of returning a string to the browser, Flask can use a template file with the `render_template` function.  
Template files must be under a `templates` sub-folder (this name is mandatory).

In [ ]:
@app.route('/home')
def home_handler():
    return render_template('home.html')

The Flask templates use the Jinja web templating engine.  
A template can inherit from another one, and contain some blocks that children templates can customize.  

Some common Jinja tags are :
- `{% block xxx %} ... {% endblock %}` to create a block that a child template can populate.
- `{% for x in my_list xxx %} ... {% endfor %}` to loop on all items of a Python list given as parameter to the template.
- `{% if condition %} ... {% else %} ... {% endif %}` to conditionally add some content in the template.


For example a `base.html` template for HTML pages could be :

In [ ]:
<!DOCTYPE HTML>
<html>
<head>
    <meta charset="utf-8" />
    <title> {% block title %} {% endblock %} </title>
</head>
<body>
    {% block body %} {% endblock %}
</body>
</html>

And a child template could extend this base template and populate the `title` and `body` blocks :

In [ ]:
{% extends 'base.html' %}

{% block title %}
Home
{% endblock %}

{% block body %}
<h1>Home page</h1>
{% endblock %}

Templates can receive parameters from the Python code.  
We can send a list of dict, and then loop on this list in the template with the `{% for x in my_list %}` Jinja tag.  
Some Python code can also be included in the template with the `{{ python code }}` notation.

_Flask handler :_

In [ ]:
@app.route('/friends')
def friends_handler():
    all_friends = [
        {'name': 'Bruce', 'hobby': 'tennis'},
        {'name': 'Peter', 'hobby': 'biology'},
        {'name': 'Damian', 'hobby': 'kung-fu', 'age': 12},
    ]
    return render_template('friends.html', friends=all_friends)

_friends.html template :_

In [ ]:
{% block body %}
<h2>Friends list :</h2>
<ul>
{% for friend in friends %}
    <li> {{ friend.name }} 
        {% if friend.age %}({{friend.age}} years old) {% endif %}
        who likes {{ friend.hobby }}
    </li>
{% endfor %}
</ul>
{% endblock %}

### Static files

Flask can serve static content, it needs to be added under a `static` sub-folder.  
The `static` sub-folder can then have a deeper hierarchy (for example contain folders for css, javascript, image files...)

Files under the `static` folder can be referenced from a template, for example :

In [ ]:
<link rel="stylesheet" href="static/css/app.css" />

The recommended way is to use the `url_for()` Flask method instead, that allows runtime parameters :

In [ ]:
<link rel="stylesheet" href="{{ url_for('static', filename='css/app.css') }}" />

### Database integration

Flask offers a nice integration with database layers via the `flask_sqlalchemy` module.  
SQLAlchemy lets us choose an SQL database technology (SQLite, mySQL, Postgres...), define the schema of our database and easily interact with the data using the _data mapper pattern_ (using classes as handlers to database tables).

- Install with `pip install flask_sqlalchemy`
- In the code set the `SQLALCHEMY_DATABASE_URI` variable to the path of the database and instanciate a `SQLAlchemy` database object

In [ ]:
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///posts.db'  # 3 slashes is for relative path
db = SQLAlchemy(app)

- Define tables as classes extending `db.Model`, with a Column field for each table column.  
  We can define a foreign key in a child table, and the corresponding relationship in the parent table.  
  For example :

In [ ]:
class BlogPost(db.Model):
    # fields look like class variables but they are converted to instance variables by SQLAlchemy
    id         = db.Column(db.Integer, primary_key=True)
    title      = db.Column(db.String(100), nullable=False)
    content    = db.Column(db.Text, nullable=False)
    created_on = db.Column(db.DateTime, nullable=False, default=datetime.now)
    
    # Field defined as foreign key
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)

    def __repr__(self):
        return f'BlogPost({self.id}, {self.title}, {self.content}, {self.user_id}, {self.created_on})'


class User(db.Model):
    id            = db.Column(db.Integer, primary_key=True)
    username      = db.Column(db.String(20), unique=True, nullable=False)
    email         = db.Column(db.String(100), unique=True, nullable=False)
    image_file    = db.Column(db.String(20), nullable=False, default='default.jpg')
    password_hash = db.Column(db.String(60), nullable=False)
    
    # 1-to-many relationship : define a relationship in the parent and a foreign key in the child
    # calling user.posts will actually run a query to fetch the posts
    # calling post.author will get the User object associated to the foreign key
    # the 'lazy' will load the posts for a user only when requested
    posts = db.relationship('Post', backref='author', lazy=True)

    def __repr__(self):
        return f'User({self.id}, {self.username}, {self.email}, {self.image_file})'

- the database can be created once before we run the website with the `create_all()` method.  
  It reads all model classes defined in the file and creates the corresponding tables.  
  For SQLite, it creates a `posts.db` database file in the current folder.

In [ ]:
from app import db
db.create_all()   # create all tables defined as classes in the app.py file

- The database can then be accessed and populated by the web server URL handlers by using instances of the model classes.  
  For example, basic CRUD operations can be performed with :

In [ ]:
# read all blog posts in DB
res = BlogPost.query.all()

# filter the result
res = BlogPost.query.filter_by(title='Blog post 1').all()

# create a user in DB
db.session.add(User(username='user1', email='user1@blog.com', password_hash='hash1'))
db.session.commit()

# create a blog post in DB
db.session.add(BlogPost(title='Blog post 1', content='This is the content', user_id=1))
db.session.commit()

# get a blog post by ID
res = BlogPost.query.get(1)

# use pseudo-fields of the relationship just like normal fields
res = User.query.get(1).posts        # actually perform a query to retrieve the posts
res = BlogPost.query.get(1).author

# update a blog post by ID
post = BlogPost.query.get(1)
post.title += '!!!'
db.session.commit()

# delete a blog post by ID
db.session.delete(BlogPost.query.get(1))
db.session.commit()

### Bootstrap style

A webapp served with Flask can, just like any other webapp, use Bootstrap styles to improve its UI.  
As described on Bootstrap website https://getbootstrap.com, it only requires to add a `<link>` tag in the header and one or more `<script>` tag(s) at the bottom of the body to make all Bootstrap styles available to our web pages.

In the _Examples_ tab, the Bootstrap website offers many ready-to-use code snippets to include nice looking components in our pages, such as :
- Container block style with `class="container"`
- Button style with `class="btn btn-primary"`
- Navbar components with the `<nav>` tag
- Form components (input, text area, dropdown...)


### Forms with Flask


#### Native HTML Forms

It is possible to use forms only with standard HTML `input` fields and a `button` for submission :


In [ ]:
<form action="/posts/create" method="POST">
    <label for="title">Title</label>
    <input class="form-control" type="text" placeholder="Enter title" name="title" id="title">
    <br />
    <label for="content">Content</label>
    <textarea class="form-control" placeholder="Enter your post content" name="content" id="content"></textarea>
    <br />
    <input class="btn btn-primary" type="submit" value="Submit" />
</form>

When clicking on the Submit button, a POST request is sent to the specified URL, and the user values can be accessed in the Python handler with :

In [ ]:
post_title = request.form['title']
post_content = request.form['content']

#### Flask WT forms module

Flask can help with the form's fields and validation using the WT forms module.  
It can be installed with :

In [ ]:
pip install flask-wtf 
pip install email_validator

A Python class is created for each form with the fields and validators for each field.  
The form object is then given to the HTML template to easily generate the HTML code and validate the input :

In [ ]:
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField
from wtforms.validators import DataRequired, Length, Email, EqualTo


# Form for the registration of a new user
class SignupForm(FlaskForm):
    username         = StringField('Username', validators=[DataRequired(), Length(min=5, max=20)])
    email            = StringField('Email', validators=[DataRequired(), Email()])
    password         = PasswordField('Password', validators=[DataRequired(), Length(min=6)])
    confirm_password = PasswordField('Confirm Password', validators=[DataRequired(), EqualTo("password")])
    remember         = BooleanField('Remember me')
    submit           = SubmitField('Sign up')

The handler can then use this class to send to an HTML template :

In [ ]:
@app.route('/signup', methods=['GET', 'POST'])
def signup_handler():
    form = SignupForm()
    if form.validate_on_submit():
        flash(f'Account created for {form.username.data}!', 'success')
        return redirect(url_for('home_handler'))
    return render_template('signup.html', form=form)

This form can then be used by the template to generate the HTML tags and access the validation status.  
For example, the below form creates a label and an input for the username field, and displays any potential error.  
It uses some Bootstrap classes for the components style and the errors highlighting.

In [ ]:
<!-- empty action will post to the same route -->
<form method="POST" action="">
    <!-- required for security -->
    {{ form.hidden_tag() }}
    <fieldset class="form-group">

        <!-- Username field -->
        <div class="form-group">
            {{ form.username.label(class="form-control-label") }}
            {% if form.username.errors %}
                {{ form.username(class="form-control form-control-lg is-invalid") }}
                <div class="invalid-feedback">
                    {% for error in form.username.errors %}
                        <span>{{ error }}</span>
                    {% endfor %}
                </div>
            {% else %}
                {{ form.username(class="form-control form-control-lg") }}
            {% endif %}
        </div>

        [ ... same for other fields ...]

    </fieldset>
    <div class="form-group mt-3">
        {{ form.submit(class="btn btn-outline-info") }}
    </div>
</form>